# Web application

> Module that create the web application to make it possible to upload a slX-file and run the conversion from the slX-files to shape, csv and geopackage

## Imports and setting up of module

In [ ]:
#| export
#| default_exp web#| export
#| default_exp web

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import sys


In [ ]:
sys.path.append('/app/data/sonar2csv_shape/')

In [ ]:
#| export
from fastcore.utils import *
from fasthtml.common import *
from fasthtml.jupyter import *
from fastlite import *
import fasthtml.components as fh
import httpx
from pathlib import Path

from sonar2csv_shape.convert import *

In [ ]:
process_sonar_file??

Signature:
process_sonar_file(
    sl_filepath: pathlib.Path,
    folder_out: pathlib.Path,
    to_crs: str = 'epsg:28992',
) -> None
Source:   
def process_sonar_file(
    sl_filepath: Path, # Path to sl2 or sl3 file
    folder_out: Path, # Output folder for exported files
    to_crs: str = "epsg:28992" # Target CRS
    ) -> None:
    "Process sonar file: extract height, convert to GeoDataFrame, clean, and export"
    msrmnt_height = extract_height(sl_filepath)
    gdf = slx2gdf(sl_filepath, to_crs=to_crs)
    gdf_clean = clean_gdf(gdf, msrmnt_height)
    export_gdf(gdf_clean, sl_filepath.stem, folder_out)
File:      ~/code/sonar2csv_shape/sonar2csv_shape/core.py
Type:      function

## Setting styling of Webapp

In [ ]:

daisy_hdrs = (
    Link(href='https://cdn.jsdelivr.net/npm/daisyui@5', rel='stylesheet', type='text/css'),
    Script(src='https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4'),
    Link(href='https://cdn.jsdelivr.net/npm/daisyui@5/themes.css', rel='stylesheet', type='text/css')
)

In [ ]:
app = FastHTML(hdrs=daisy_hdrs)
rt = app.route

In [ ]:
# import dialoghelper as dh

In [ ]:
# dir(dh)

In [ ]:
# dh.fc_tool_info()

In [ ]:
from fastcore.tools import *

Tools available from `fastcore.tools`:

- &`rg`: Run the `rg` command with the args in `argstr` (no need to backslash escape)
- &`sed`: Run the `sed` command with the args in `argstr` (e.g for reading a section of a file)
- &`view`: View directory or file contents with optional line range and numbers
- &`create`: Creates a new file with the given content at the specified path
- &`insert`: Insert new_str at specified line number
- &`str_replace`: Replace first occurrence of old_str with new_str in file
- &`strs_replace`: Replace for each str pair in old_strs,new_strs
- &`replace_lines`: Replace lines in file using start and end line-numbers

In [ ]:
def get_preview(app):
    return partial(HTMX, app=app, host=None, port=None)
preview = get_preview(app)

In [ ]:
partial?

Init signature: partial(self, /, *args, **kwargs)
Docstring:     
partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.
File:           ~/.local/share/uv/python/cpython-3.11.12-linux-x86_64-gnu/lib/python3.11/functools.py
Type:           type
Subclasses:     

Can you please explain to me what the `partial` function does, how it does that and what its purpose is?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

The `partial` function is a really useful tool from Python's `functools` module. Let me break it down for you:

**What it does:** `partial` takes a function and "freezes" some of its arguments, creating a new function with those arguments pre-filled.

**How it works:** When you call `partial(func, arg1, arg2)`, it returns a new function that, when called, will automatically pass `arg1` and `arg2` to `func`, along with any additional arguments you provide later.

**Simple example:**
```python
def greet(greeting, name):
    return f"{greeting}, {name}!"
```

If you use `partial(greet, "Hello")`, you get a new function that already has "Hello" as the greeting, so you only need to provide the name later.

In your code, `partial(HTMX, app=app, host=None, port=None)` creates a new function where `app`, `host`, and `port` are already set. So when you call `preview(...)` later, those parameters are automatically included.

Does this make sense so far? Would you like me to show you a quick example of how `partial` works in practice?

In [ ]:
@rt
def testing(): return Div(P('hi'), style='color:red')

In [ ]:
srv = JupyUvi(app=app)

In [ ]:
preview("testing")

In [ ]:
@rt
def proc_btn(): return f'hi'

@rt
def testbtn():
    return Div(
        Button('click me', hx_post=proc_btn, id='btn', hx_target='#test', cls='btn'),
        Div(id='test')
    )

In [ ]:
preview("testbtn")

## Create upload

We grabbed the logic from [FastHTML docs - quickstart](https://fastht.ml/docs/tutorials/quickstart_for_web_devs.html#single-file-uploads)

In [ ]:

upload_dir = Path("/app/data")

@rt
def get():
    return Titled("File Upload Demo",
        Article(
            Form(hx_post=upload, hx_target="#result-one")(
                Input(type="file", name="file"),
                Button("Upload", type="submit", cls='btn'),
            ),
            Div(id="result-one")
        )
    )

def FileMetaDataCard(file):
    return Article(
        Header(H3(file.filename)),
        Ul(
            Li('Size: ', file.size),            
            Li('Content Type: ', file.content_type),
            Li('Headers: ', file.headers),
        )
    )    

@rt
async def upload(file: UploadFile):
    card = FileMetaDataCard(file)
    filebuffer = await file.read()
    (upload_dir / file.filename).write_bytes(filebuffer)
    return card

In [ ]:
preview("get")

## Create download

> I asked the SolveIt Discord Buddy how to download files with FastHTML. This was Buddy's response.

For file downloads in FastHTML, create routes that return `FileResponse` with the generated files. Here's the pattern:

```python
from fasthtml.common import *
from starlette.responses import FileResponse

app = FastHTML()

@rt("/download/{filename}")
def get(filename:str):
    filepath = f"/app/data/{filename}"
    return FileResponse(filepath, filename=filename)

# In your upload handler, after processing:
@rt("/process")
async def post(file: UploadFile):
    # Your magic happens here, creates 6 files
    # Return links to download them
    links = [A(f"Download {i}", href=f"/download/file{i}.txt") for i in range(6)]
    return Div(*links)
```

For multiple files at once, you can create a zip file and serve that, or use HTMX to trigger multiple downloads. The key is using `FileResponse` from Starlette (included with FastHTML) which handles the browser download prompt automatically.


Lets write a page with a download button. When I hit that button it downloads a file from the `/app/data` folder. Use the information from the above note to know how to do this.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Here's a page with a download button:

```python
from starlette.responses import FileResponse

@rt("/download/{filename}")
def get_file(filename:str):
    filepath = upload_dir / filename
    return FileResponse(filepath, filename=filename)

@rt("/download_page")
def get():
    return Titled("File Download Demo",
        Article(
            H3("Download a file"),
            A("Download File", href="/download/test.txt", cls='btn')
        )
    )
```

You'll need to have a file called `test.txt` in your `/app/data` folder to test this. Want me to help you create a test file, or would you like to modify this to download a specific file you already have?

In [ ]:
from starlette.responses import FileResponse


In [ ]:

@rt
def get_file():
    filepath = "/app/data/solveit_settings.json" # filename
    return FileResponse(filepath, filename="test.txt")

@rt("/download_page")
def get():
    return Titled("File Download Demo",
        Article(
            H3("Download a file"),
            A("Download File", href="/get_file", cls='btn')
        )
    )

In [ ]:
preview("download_page")

The response from `get_file` place the text in the button instead of downloading it to the computer if I use `hx_get`. If I use `href`. Nothing happens.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

The issue is that HTMX intercepts the request and puts the response in the page, while `href` on a `Button` doesn't work. Use an `A` tag instead:

```python
@rt
def get_file():
    filepath = "/app/data/solveit_settings.json"
    return FileResponse(filepath, filename="test.txt")

@rt("/download_page")
def get():
    return Titled("File Download Demo",
        Article(
            H3("Download a file"),
            A("Download File", href="/get_file", cls='btn')
        )
    )
```You need to use a regular link (`A`) instead of a `Button` for downloads, since HTMX intercepts the request. Here's the fix:

```python
@rt("/download_page")
def get():
    return Titled("File Download Demo",
        Article(
            H3("Download a file"),
            A("Download File", href="/get_file", cls='btn')
        )
    )
```

The `Button` element doesn't support `href` - that's an `A` (anchor/link) attribute. Using `A` with `href` will trigger a proper browser download instead of an HTMX request.

## Starting the process

Now create a web application that has the upload a file on the index page. When the file is done uploading it should use the function `process_sonar_file` from the module `core` to convert that sl2 or sl3 file to the shape en csv and geopackage files and store the result in a single zip file in the folder `/app/data`. Then this file must be downloaded to the users PC. Finally the uploaded file and the created and downloaded files must be removed from `/app/data`. Use the examples off uploading and downloading as given above.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

```python
import zipfile, os
from starlette.responses import FileResponse

@rt("/")
def get():
    return Titled("Sonar File Converter",
        Article(
            H3("Upload sl2 or sl3 file"),
            Form(hx_post=upload_convert, hx_target="#result")(
                Input(type="file", name="file", accept=".sl2,.sl3"),
                Button("Upload & Convert", type="submit", cls='btn'),
            ),
            Div(id="result")
        )
    )

@rt
async def upload_convert(file: UploadFile):
    filebuffer = await file.read()
    input_path = upload_dir / file.filename
    input_path.write_bytes(filebuffer)
    
    process_sonar_file(input_path, upload_dir)
    
    zip_name = f"{input_path.stem}.zip"
    zip_path = upload_dir / zip_name
    with zipfile.ZipFile(zip_path, 'w') as zf:
        for f in upload_dir.glob(f"{input_path.stem}.*"):
            if f.suffix != '.zip' and f != input_path: zf.write(f, f.name)
    
    for f in upload_dir.glob(f"{input_path.stem}.*"):
        if f != zip_path: f.unlink()
    
    return Div(
        P("Conversion complete!"),
        A("Download Results", href=f"/download_zip/{zip_name}", cls='btn')
    )

@rt("/download_zip/{filename}")
def get(filename:str):
    filepath = upload_dir / filename
    response = FileResponse(filepath, filename=filename)
    filepath.unlink()
    return response
```

How can I add some feedback so that the user can see that the application is running and doing the conversion?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You can add a loading indicator using HTMX's `hx-indicator` attribute:

```python
@rt("/")
def get():
    return Titled("Sonar File Converter",
        Article(
            H3("Upload sl2 or sl3 file"),
            Form(hx_post=upload_convert, hx_target="#result", hx_indicator="#spinner")(
                Input(type="file", name="file", accept=".sl2,.sl3"),
                Button("Upload & Convert", type="submit", cls='btn'),
            ),
            Div(id="spinner", cls="htmx-indicator")("Processing..."),
            Div(id="result")
        )
    )
```

Or for a more visible progress message, you can return intermediate updates:

```python
@rt
async def upload_convert(file: UploadFile):
    filebuffer = await file.read()
    input_path = upload_dir / file.filename
    input_path.write_bytes(filebuffer)
    
    yield Div(P("Converting sonar file..."), cls="alert alert-info")
    
    process_sonar_file(input_path, upload_dir)
    
    yield Div(P("Creating zip file..."), cls="alert alert-info")
    
    zip_name = f"{input_path.stem}.zip"
    zip_path = upload_dir / zip_name
    with zipfile.ZipFile(zip_path, 'w') as zf:
        for f in upload_dir.glob(f"{input_path.stem}.*"):
            if f.suffix != '.zip' and f != input_path: zf.write(f, f.name)
    
    for f in upload_dir.glob(f"{input_path.stem}.*"):
        if f != zip_path: f.unlink()
    
    yield Div(P("Conversion complete!"), A("Download Results", href=f"/download_zip/{zip_name}", cls='btn'))
```

Which approach would you prefer?

In [ ]:
import zipfile, os
from starlette.responses import FileResponse


In [ ]:

upload_dir = Path("/app/data")

@rt
def home():
    return Titled("Sonar File Converter",
        Article(
            H3("Upload sl2 or sl3 file"),
            Form(hx_post=upload_convert, hx_target="#result")(
                Input(type="file", name="file", accept=".sl2,.sl3"),
                Button("Upload & Convert", type="submit", cls='btn'),
            ),
            Div(id="result")
        )
    )

@rt
async def upload_convert(file: UploadFile):
    filebuffer = await file.read()
    input_path = upload_dir / file.filename
    input_path.write_bytes(filebuffer)
    
    process_sonar_file(input_path, upload_dir)
    
    zip_name = f"{input_path.stem}.zip"
    zip_path = upload_dir / zip_name
    with zipfile.ZipFile(zip_path, 'w') as zf:
        for f in upload_dir.glob(f"{input_path.stem}.*"):
            if f.suffix != '.zip' and f != input_path: zf.write(f, f.name)
    
    for f in upload_dir.glob(f"{input_path.stem}.*"):
        if f != zip_path: f.unlink()
    
    return Div(P("✓ File uploaded successfully")) # , P("✓ Conversion successful"), P("✓ Zip file created"), A("Download Results", href=f"/get/{zip_name}", cls='btn', download=zip_name), P("The file will be downloaded to your downloads folder"), id="result")

# @rt
# def home():
#     return Titled("Sonar File Converter",
#         Article(
#             H3("Upload sl2 or sl3 file"),
#             Form(hx_post=upload_convert, hx_target="#result", hx_indicator="#loading")(
#                 Input(type="file", name="file", accept=".sl2,.sl3"),
#                 Button("Upload & Convert", type="submit", cls='btn'),
#             ),
#             Div(id="loading", cls="htmx-indicator loading loading-spinner loading-lg"),
#             Div(id="result")
#         ),
#         Style(".htmx-indicator{display:none} .htmx-request .htmx-indicator{display:inline-block} .htmx-request.htmx-indicator{display:inline-block}")
#     )

# @rt
# async def upload_convert(file: UploadFile):
#     filebuffer = await file.read()
#     input_path = upload_dir / file.filename
#     input_path.write_bytes(filebuffer)
    
#     process_sonar_file(input_path, upload_dir)
    
#     zip_name = f"{input_path.stem}.zip"
#     zip_path = upload_dir / zip_name
#     with zipfile.ZipFile(zip_path, 'w') as zf:
#         for f in upload_dir.glob(f"{input_path.stem}.*"):
#             if f.suffix != '.zip' and f != input_path: zf.write(f, f.name)
    
#     for f in upload_dir.glob(f"{input_path.stem}.*"):
#         if f != zip_path: f.unlink()
    

@rt
def get(filename:str):
    filepath = upload_dir / filename
    response = FileResponse(filepath, filename=filename)
    filepath.unlink()
    return response

In [ ]:
preview(home)

I get a status "101" when I hit the Upload&Convert button.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Status 101 means "Switching Protocols" - this happens when FastHTML tries to use Server-Sent Events (SSE) for the `yield` statements, but the setup isn't quite right. 

Try adding `hx_ext='sse'` and using SSE properly, or simplify by removing the intermediate yields:

```python
@rt
async def upload_convert(file: UploadFile):
    filebuffer = await file.read()
    input_path = upload_dir / file.filename
    input_path.write_bytes(filebuffer)
    
    process_sonar_file(input_path, upload_dir)
    
    zip_name = f"{input_path.stem}.zip"
    zip_path = upload_dir / zip_name
    with zipfile.ZipFile(zip_path, 'w') as zf:
        for f in upload_dir.glob(f"{input_path.stem}.*"):
            if f.suffix != '.zip' and f != input_path: zf.write(f, f.name)
    
    for f in upload_dir.glob(f"{input_path.stem}.*"):
        if f != zip_path: f.unlink()
    
    return Div(P("Conversion complete!"), A("Download Results", href=f"/get/{zip_name}", cls='btn'))
```

This removes the `yield` statements and just returns the final result. The conversion will still happen, but without the intermediate progress updates.

But I want to give the user some feedback that something is actually happening after the user clicked the upload and convert button. Sometimes the conversion takes almost a minute. The user will than think that the app is broken if I don't give some visual feedback the process is succesfully running.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You can add a loading spinner that shows immediately when the button is clicked. Here's how:

```python
@rt
def home():
    return Titled("Sonar File Converter",
        Article(
            H3("Upload sl2 or sl3 file"),
            Form(hx_post=upload_convert, hx_target="#result", hx_indicator="#loading")(
                Input(type="file", name="file", accept=".sl2,.sl3"),
                Button("Upload & Convert", type="submit", cls='btn'),
            ),
            Div(id="loading", cls="htmx-indicator loading loading-spinner loading-lg"),
            Div(id="result")
        ),
        Style(".htmx-indicator{display:none} .htmx-request .htmx-indicator{display:inline-block} .htmx-request.htmx-indicator{display:inline-block}")
    )
```

The spinner will automatically show while the request is processing and hide when complete. The `hx-indicator` attribute tells HTMX which element to show during the request.

The spinner is nice. But it can still take a very long time. It would be good if I can show that the file is succesfully uploaded. Starting conversion. This can take up to two minutes. The conversion is succesfull. A zipfile will be downloaded to you downloads folder.

or something like that

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You can use HTMX's `hx-swap-oob` (out of band swaps) to update the status message multiple times during the process:

```python
@rt
def home():
    return Titled("Sonar File Converter",
        Article(
            H3("Upload sl2 or sl3 file"),
            Form(hx_post=upload_convert, hx_target="#result")(
                Input(type="file", name="file", accept=".sl2,.sl3"),
                Button("Upload & Convert", type="submit", cls='btn'),
            ),
            Div(id="result")
        )
    )

@rt
async def upload_convert(file: UploadFile):
    filebuffer = await file.read()
    input_path = upload_dir / file.filename
    input_path.write_bytes(filebuffer)
    
    yield Div(P("✓ File uploaded successfully"), P("Starting conversion... This can take up to two minutes.", cls="loading loading-spinner loading-sm"), id="result")
    
    process_sonar_file(input_path, upload_dir)
    
    yield Div(P("✓ File uploaded successfully"), P("✓ Conversion successful"), P("Creating zip file..."), id="result")
    
    zip_name = f"{input_path.stem}.zip"
    zip_path = upload_dir / zip_name
    with zipfile.ZipFile(zip_path, 'w') as zf:
        for f in upload_dir.glob(f"{input_path.stem}.*"):
            if f.suffix != '.zip' and f != input_path: zf.write(f, f.name)
    
    for f in upload_dir.glob(f"{input_path.stem}.*"):
        if f != zip_path: f.unlink()
    
    yield Div(P("✓ File uploaded successfully"), P("✓ Conversion successful"), P("✓ Zip file created"), A("Download Results", href=f"/get/{zip_name}", cls='btn', download=zip_name), P("The file will be downloaded to your downloads folder"), id="result")
```

This will update the status message at each step of the process.

In [ ]:
srv.stop()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()